In [8]:
import numpy as np
import pandas as pd
import csv
from numpy import linspace,exp
from numpy.random import randn
# import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
# import seaborn as sns

import scipy 
import editdistance
import sklearn.metrics
import statsmodels.api as sm
from hmmlearn import hmm

In [9]:
##  Convert the loaded JSB_Chorales into the a list of singular notes to match format of midi notes.
def get_training_notes(dataset):
    notes=[]
    current_notes=[]   
    
    for piece in dataset['train']:
        for seq in piece:
            unwanted=[]
            for i in current_notes:
                if i not in list(seq):
                    notes.append(i)
                    unwanted.append(i)
            current_notes = [ele for ele in current_notes if ele not in unwanted] 

            for j in list(seq):
                if j not in current_notes:
                    notes.append(j)
                    current_notes.append(j)
    return notes      

In [10]:
def find_vel(newNotes, velocity,gp):
    # Use splines to interpolate the velocities
    newVelocities = np.zeros(len(newNotes))
    y = velocity[np.nonzero(velocity)] # all the nonzero elements need to be interpolated
    indicies = []
    for i in np.unique(newNotes):
        indicies.append(np.where(newNotes == i)[0][::2])  ## set every other pitch occurrence to 0 (turn off)
   
    unlist = [item for sublist in indicies for item in sublist]
    unlist.sort()
    
    X = np.array(range(0,len(y)))
    s = UnivariateSpline(X, y, s=300) #750
    xs = np.linspace(0, len(y), len(unlist), endpoint = True)
    ys = s(xs)   
    newVelocities[np.array(unlist)] = np.round(ys).astype(int)
    #Fix entries that are too small or too large due to spline overfitting
    newVelocities[np.where(newVelocities < 0)[0]] = y[-1]
    #print(y[-1])
    newVelocities = newVelocities.astype(int)     
    return(newVelocities)

In [11]:
class pre_process(object):
    def __init__(self, input_filename, min_note):
        self.input_filename = input_filename
        self.min_note = min_note
      
    
    def read_process(self):
        with open(self.input_filename,encoding = "ISO-8859-1") as fd:
            reader=csv.reader(fd)
            rows= [row for idx, row in enumerate(reader)]
        song = pd.DataFrame(rows)
        r,c = np.where(song == ' Header')
        quarter_note = song.iloc[r,5].values.astype(int)[0]
        r, c = np.where(song == ' Time_signature')
        num = song.iloc[r, 3].values.astype(int)[0]
        denom = song.iloc[r, 4].values.astype(int)[0]**2
        try:
            r, c = np.where(song == ' Key_signature')
            key = song.iloc[r,3].values.astype(int)[0]
        except:
            key = None
        
        song_model = song[song.iloc[:, 2].isin([' Note_on_c', ' Note_off_c'])]
        song_model = song_model.loc[song_model.iloc[:,0] == np.max(song_model.iloc[:,0])]
        
        time = np.array(song_model.iloc[:,1]).astype(int)
        notes = np.array(song_model.iloc[:,4]).astype(int)
        velocity = np.array(song_model.iloc[:,5]).astype(int)
        measures = np.round(np.max(time)/quarter_note)/num
        min_note = quarter_note
        actual = np.arange(0, min_note*measures*num, min_note).astype(int) 
        time = np.array([find_nearest(actual, time[i]) for i in range(len(time))]).astype(int)
        return (quarter_note, num, denom, key, measures, time, notes, velocity, song, song_model.index)

In [12]:
from sklearn import preprocessing

## Convert from pitch representation (integers 0-127) to integers (0-max)
## x is the input vector of notes and code is a vector of the unique pitches in x
class LabelEncoderDecoder():
    def __init__(self, x):
        self.processor = preprocessing.LabelEncoder()
        self.processor.fit(x)
        
    def encode(self, y):
        return self.processor.transform(y)
    
    def decode(self, y):
        return self.processor.inverse_transform(y)
    
    def all_classes(self):
        return self.processor.classes_
    
    def all_encode_classes(self):
        return self.encode(self.all_classes())
    
    def max_encode_class(self):
        return max(self.all_encode_classes())
    

## Function to convert the values in array to the nearest values in the array value
## Used to convert continues TVAR generated pitches to closest integer values for MIDI representation
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [13]:
def output_music_csv(output_filename,time,newNotes,newVelocities):
    song.iloc[ind, 1] = time
    song.iloc[ind, 4] = newNotes
    song.iloc[ind, 5] = newVelocities
    song.iloc[ind[np.where(newVelocities !=0)], 2] = ' Note_on_c'
    song.iloc[ind[np.where(newVelocities ==0)], 2] = ' Note_off_c'
    split = output_filename.split('.')
    song.to_csv(output_filename, header = None, index = False)

In [14]:
def generate_output_base(model, n):
    start_prob = model.startprob_
    transmat = model.transmat_
    emissionprob = model.emissionprob_
    m = transmat.shape[0]
    k = emissionprob.shape[1]
    zstates = np.arange(0, m, dtype = int)
    xstates = np.arange(0, k, dtype = int)
    z = np.zeros(n, dtype = int)
    x = np.zeros(n, dtype = int)
    z[0] = np.random.choice(zstates, size = 1, p = start_prob)
    for j in range(1, n):
        z[j] = np.random.choice(zstates, size = 1, p = transmat[z[j-1], :])
    for i in range(0, n):
        x[i] = np.random.choice(xstates, size = 1, p = emissionprob[z[i], :])
    return x

In [15]:
def generate_output_layered(emissionprob1, emissionprob2, emissionprob3, zStar3, n):
    m = emissionprob1.shape[0]
    k = emissionprob1.shape[1]
    zstates = np.arange(0, m, dtype = int)
    xstates = np.arange(0, k, dtype = int)
    output = np.zeros(shape = (3,n), dtype = int)
    for j in range(0,n):
        output[2, j] = np.random.choice(zstates, size = 1, p = emissionprob3[zStar3[j], :])
        output[1, j] = np.random.choice(zstates, size = 1, p = emissionprob2[output[2, j], :])
        output[0, j] = np.random.choice(xstates, size = 1, p = emissionprob1[output[1, j], :])
    return output[0,:]

In [16]:
def generate_output_interpolate(model1, model2, n, gamma, leds):
    state_num = model1.emissionprob_.shape[0]
    start_prob = model1.startprob_ * gamma + model2.startprob_ * (1-gamma)
    transmat = model1.transmat_ * gamma + model2.transmat_ * (1-gamma)
    specific = leds[0].decode(np.arange(model1.emissionprob_.shape[1]))
    generic = leds[1].decode(np.arange(model2.emissionprob_.shape[1]))
    emissionprob = np.zeros([state_num, leds[2].max_encode_class() + 1])
    print(emissionprob.shape)
    for j in range(len(specific)):
        col = leds[2].encode([specific[j]])[0]
        emissionprob[:, col] += gamma * model1.emissionprob_[:, j]
    for j in range(len(generic)):
        col = leds[2].encode([generic[j]])[0]
        emissionprob[:, col] += (1-gamma) * model2.emissionprob_[:, j]
        
    m = transmat.shape[0]
    k = emissionprob.shape[1]
    zstates = np.arange(0, m, dtype = int)
    xstates = np.arange(0, k, dtype = int)
    z = np.zeros(n, dtype = int)
    x = np.zeros(n, dtype = int)
    z[0] = np.random.choice(zstates, size = 1, p = start_prob)
    for j in range(1, n):
        z[j] = np.random.choice(zstates, size = 1, p = transmat[z[j-1], :])
    for i in range(0, n):
        x[i] = np.random.choice(xstates, size = 1, p = emissionprob[z[i], :])
    return x

<h1>Base HMM</h1>

In [ ]:
# CSV to Midi
import py_midicsv


# Save the parsed MIDI file to disk
with open(dir_path+fname+'_hmm_layered'+str(states)+".midi", "wb") as output_file:
    midi_writer = py_midicsv.FileWriter(output_file)
    midi_writer.write(midi_object)

In [254]:
import os
# Choose a music template
fname = 'twinkle-twinkle-little-star'
min_note = 1024

# load music template from midi file
dir_path = './data/'
filepath = os.path.join(dir_path,fname+'.csv')
music = pre_process(filepath,min_note)

# Obtain all components of a midi piano file
quarter_note, num, denom, key, measures, time, \
            notes_template, velocity, song, ind = music.read_process()
possibleVelocities =  np.unique(velocity)

tr_notes = notes_template

In [20]:
a = set()
a.add(1)
b = set()
b.add(2)
a.union(b)

{1, 2}

In [257]:
# Preprocess training data
possibleNotes = np.unique(tr_notes)
k = len(possibleNotes)
led = LabelEncoderDecoder(tr_notes)
xNotes = led.encode(tr_notes)
n = len(xNotes)
states = 10

# Train with HMM
model = hmm.MultinomialHMM(n_components=states)
observ_data = xNotes.reshape(len(xNotes),1)
model.fit(observ_data)
newNotes = led.decode(generate_output_base(model, len(notes_template)))
newVelocities = find_vel(newNotes, velocity, gp = False)

# Output music into csv 
output_filename = dir_path+fname+'_hmm'+str(states)+'.csv'
print(output_filename)
output_music_csv(output_filename,time,newNotes.astype(int),newVelocities)

./data/chopin_hmm10.csv


In [258]:
# CSV to Midi
import py_midicsv

with open(output_filename, "r") as f:
    midi_object = py_midicsv.csv_to_midi(f.readlines())

# Save the parsed MIDI file to disk
with open(dir_path+fname+'_hmm'+str(states)+".midi", "wb") as output_file:
    midi_writer = py_midicsv.FileWriter(output_file)
    midi_writer.write(midi_object)

<h1> Layered HMM </h1>

In [261]:
model = hmm.MultinomialHMM(n_components=states)
observ_data = xNotes.reshape(len(xNotes),1)
model.fit(observ_data)
zStar1 = model.predict(observ_data).astype(int)
emissionprob1 = model.emissionprob_

model2 = hmm.MultinomialHMM(n_components=states)
observ_data = zStar1.reshape(len(zStar1),1)
model2.fit(observ_data)
zStar2 = model2.predict(observ_data).astype(int)
emissionprob2 = model2.emissionprob_

model3 = hmm.MultinomialHMM(n_components=states)
observ_data = zStar2.reshape(len(zStar2),1)
model3.fit(observ_data)
zStar3 = model3.predict(observ_data).astype(int)
emissionprob3 = model3.emissionprob_

In [263]:
newNotes = led.decode(generate_output_layered(emissionprob1, emissionprob2, emissionprob3, zStar3, n))
newVelocities = find_vel(newNotes, velocity, gp = False)

# Output music into csv 
output_filename = dir_path+fname+'_hmm_layered'+str(states)+'.csv'
print(output_filename)
output_music_csv(output_filename,time,newNotes.astype(int),newVelocities)

./data/chopin_hmm_layered10.csv


In [264]:
# CSV to Midi
import py_midicsv

with open(output_filename, "r") as f:
    midi_object = py_midicsv.csv_to_midi(f.readlines())

# Save the parsed MIDI file to disk
with open(dir_path+fname+'_hmm_layered'+str(states)+".midi", "wb") as output_file:
    midi_writer = py_midicsv.FileWriter(output_file)
    midi_writer.write(midi_object)

<h1> HMM Interpolation </h1>

In [265]:
# Preprocess training data for specific HMM
# Choose a music template
fname = 'twinkle-twinkle-little-star'
min_note = 1024

# load music from midi file
dir_path = './data/'
filepath = os.path.join(dir_path,fname+'.csv')
music = pre_process(filepath,min_note)

# Obtain all components of a midi piano file
quarter_note, num, denom, key, measures, time, \
            tr_notes_specific, velocity, song, ind = music.read_process()
possibleVelocities =  np.unique(velocity)

# Preprocess training data for generic HMM
pickle_file_path = r".\JSB_Chorales.pickle"
file = open(pickle_file_path, 'rb')
dataset = pickle.load(file)
file.close()
tr_notes_generic = get_training_notes(dataset)


possibleNotes = np.unique(np.concatenate([tr_notes_generic, tr_notes_specific]))

n = len(tr_notes_specific)
states = 10

# Train specific HMM
led_specific = LabelEncoderDecoder(tr_notes_specific)
model_specific = hmm.MultinomialHMM(n_components=states)
xNotes_specific = led_specific.encode(tr_notes_specific)
observ_data_specific = xNotes_specific.reshape(len(tr_notes_specific),1)
model_specific.fit(observ_data_specific)

# Train generic HMM
led_generic = LabelEncoderDecoder(tr_notes_generic)
model_generic = hmm.MultinomialHMM(n_components=states)
xNotes_generic = led_generic.encode(tr_notes_generic)
observ_data_generic = xNotes_generic.reshape(len(xNotes_generic),1)
model_generic.fit(observ_data_generic)

Fitting a model with 209 free scalar parameters with only 178 data points will result in a degenerate solution.


MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=10,
               n_iter=10, params='ste',
               random_state=RandomState(MT19937) at 0x261FF098570,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [266]:
# Interpolation
led = LabelEncoderDecoder(possibleNotes)
leds = [led_specific, led_generic, led]
newNotes = led.decode(generate_output_interpolate(model_specific, model_generic, len(tr_notes_specific), 0.5, leds))
newVelocities = find_vel(newNotes, velocity, gp = False)

# Output music into csv 
output_filename = dir_path+fname+'_hmm_interpolate'+str(states)+'.csv'
output_music_csv(output_filename, time, newNotes.astype(int),newVelocities)

(10, 52)


In [267]:
# CSV to Midi
with open(output_filename, "r") as f:
    midi_object = py_midicsv.csv_to_midi(f.readlines())

# Save the parsed MIDI file to disk
with open(dir_path+fname+'_hmm_interpolate'+str(states)+".midi", "wb") as output_file:
    midi_writer = py_midicsv.FileWriter(output_file)
    midi_writer.write(midi_object)